## Question 5

For this question I decided to explore the relationship between salaries and wages & inflation. I used CPI as my measure of inflation.

Prior to conducting any analysis, intuitively I suspected there would be a positive association between salaries and wages and CPI.

I did some very quick googling just to ensure that my approach made sense- and found two documents which further shaped the direction of my analyis. These documents are included in this repository in the `Supporting Documents` directory.

<u>Supporting Documents:</u>

1. dp09-06-Analysing wage and price dynamics in NZ
2. p20045coll4_54-Exploring Tradable and Non-Tradable Inflation in Consumer Prices


In the Abstract of the first document listed above it says:

> _We find that there is a joint causality between wages and disaggregate inflation. An increase in wage inflation forecasts an increase in non-tradable inflation._

I found on the Stats NZ website that they publish a tradable and non-tradable inflation series, and given the quote above it made sense to me to use this data for my analysis.

I decided to create two models of the form:

$$ \text{Model 1:} \ Y_1 = \beta_1X_1 + \beta_0$$

$$ \text{Model 2:} \ Y_2 = \beta_1X_1 + \beta_0$$

Where the independent variable of each model is:

$Y_1$: is New Zealand's CPI for Tradeable groups.

$Y_2$: is New Zealand's CPI for Non-tradeable groups.

and the explanatory variable of each model is:

$X_1$: is Salaries and Wages data

I thought that Simple Linear Regression was a sensible and appropriate choice, as it was of interest to me to interpret the $\beta$ coefficients, to understand the association between salaries and wages and inflation in NZ.

Given the quote above showing that previous research had showed a positive association between wages and non-tradeable inflation, I decided that I would perform a 1-tailed test, testing only for a positive association.
As I had no reference for how salaries and wages may affect Tradeable inflation, I thought a 2-sided test appropriate.

**Distinction between Tradeable and Non-Tradeable Goods and Services:**

> Goods and services whose prices are primarily influenced by world prices and international supply and demand, and whose prices are primarily determined by factors such as domestic demand and supply. Goods and services which enter international trade and satisfy the law of one price, or at an appropriate relative price that could satisfy the law of one price, can be defined as tradable. All other goods and services will be defined as non-tradable.

Further information about the CPI and how it is calculated can be found on the Stats NZ website- https://www.stats.govt.nz/topics/consumers-price-index


#### Hypothesis:

__Model 1:__

$\mathcal{H}_0: \text{There is no association between salaries and wages and the CPI for Tradeable groups in New Zealand}$ \
$\mathcal{H}_1: \text{There is an association between salaries and wages and the CPI for Tradeable groups in New Zealand}$



__Model 2:__

$\mathcal{H}_0: \text{There is no positive association between salaries and wages and the CPI for Non-tradeable groups in New Zealand}$ \
$\mathcal{H}_1: \text{There is a positive association between salaries and wages and the CPI for Non-t
radeable groups in New Zealand}$




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

SALARIES_AND_WAGES_FILENAME = "salaries_and_wages_df.xlsx"
CPI_FILENAME = "consumers-price-index-june-2024-quarter-seasonally-adjusted.csv"

In [2]:
salaries_and_wages = pd.read_excel(f'./data/{SALARIES_AND_WAGES_FILENAME}')
cpi = pd.read_csv(f'./data/{CPI_FILENAME}')